# Resnet50

In [1]:
# import the libraries as shown below
import tensorflow 
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.applications import MobileNet
from keras.applications.vgg19 import VGG19
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt


# for other models visite https://keras.io/api/applications/

In [2]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = r'D:\dowanloadss\accident-data\train'
valid_path = r'D:\dowanloadss\accident-data\val'

In [3]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

resnet =  VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [4]:
# don't train existing weights
for layer in resnet.layers:
    layer.trainable = False

In [5]:

# useful for getting number of output classes
folders = glob(r'D:\dowanloadss\accident-data\train\*')

In [6]:
# our layers - you can add more if you want
x = Flatten()(resnet.output)

In [7]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=resnet.input, outputs=prediction)

In [8]:

# view the structure of the model
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [9]:
import matplotlib.pyplot as plt


In [10]:

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [11]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)


'''

train_datagen = ImageDataGenerator(width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2,
                                   shear_range=0.2,
                                   horizontal_flip=True,
                                    rotation_range=40,
                                   fill_mode='nearest')'
'''

test_datagen = ImageDataGenerator(rescale = 1./255)

In [12]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory(r'D:\dowanloadss\accident-data\train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 3981 images belonging to 2 classes.


In [13]:
test_set = test_datagen.flow_from_directory(r'D:\dowanloadss\accident-data\val',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 198 images belonging to 2 classes.


In [14]:

# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Instructions for updating:
Please use Model.fit, which supports generators.


C:\Users\PRADEEP\anaconda3\lib\site-packages\PIL\Image.py:2942: UserWarning: image file could not be identified because WEBP support not installed
  warnings.warn(message)


UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x000002656049E360>

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
# save it as a h5 file
from tensorflow.keras.models import load_model
#model.save('model_resnet50_10.h5')

In [ ]:
y_pred = model.predict(test_set)


In [ ]:
y_pred

In [ ]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)


In [ ]:

y_pred

In [ ]:

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [ ]:
#model=load_model('model_resnet50.h5')

In [ ]:
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
np.set_printoptions(suppress=True)
#model = tensorflow.keras.models.load_model('D:/dowanloadss/converted_keras (1)new/keras_model.h5')
#model = tensorflow.keras.models.load_model('model_resnet50.h5')

In [ ]:
import cv2
frameWidth = 640
frameHeight = 480
cap = cv2.VideoCapture('D:/dowanloadss/ambulance.mp4')
cap.set(3, frameWidth)
cap.set(4, frameHeight)
cap.set(10,150)


In [ ]:
while True:
    success, img = cap.read()
    image = img
    img = cv2.resize(img,(224,224))
    img_res = np.asarray(img)
    normalized_image_array = (img_res.astype(np.float32) / 127.0) - 1
    data[0] = normalized_image_array
    prediction = model.predict(data)
    print(prediction)
    text = np.array_str(prediction)
    cv2.putText(image,text,(15,20),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,0),3)
    cv2.imshow("Result", image)
    if cv2.waitKey(1) and 0xFF == ord('q'):
        break